In [ ]:
import numpy as np

import networkx as nx
import matplotlib.pyplot as plt

import os.path as op
import sys
sys.path.append("../")

from src import regmod
from src import utils
from src import solver

## Synthetic Example

In [ ]:
import importlib
importlib.reload(solver)

In [ ]:
path_to_data = "../resources"

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

filename = "adjacency_synthetic.pkl"

adj = utils.load(op.join(path_to_data, filename))
adj -= np.diag(np.diag(adj))

axes[0].imshow(adj, cmap='gray')
axes[0].set_title('Structural connectivity')
toy_graph = nx.Graph(adj)
nx.draw(toy_graph, ax=axes[1], with_labels=True)

### Example of a forward pass

In [ ]:
max_path_depth = 4

multi_hops_design = regmod.get_path_matrices(adj, max_path_depth)

a = 0
# Computes the alpha vector as alpha_n = a^n
alpha = [a**(i+1) for i in range(max_path_depth)]
#alpha = a

design_shortest = regmod.combine_paths_matrices(multi_hops_design, alpha=alpha)
design_shortest = regmod.build_design_shortest(adj, n_subopt=1, alpha=a)
design_model = design_shortest

y_pred_mat = regmod.predict_conduction_delays(design_model, adj, invert_weights=False)

fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(20, 5))
fig.suptitle(rf"Path design matrix, $\alpha=$"+" ".join([f"{a:1.4f}" for a in list(alpha)]))
axes[0].imshow(y_pred_mat, cmap='hot')
axes[0].set_title("Conductance delays $\hat{y}$")
utils.add_cbar(fig, axes[0])
utils.annotate_heatmap(fig, axes[0], y_pred_mat, adapt_color=0.6)

axes[1].imshow(design_model, cmap='gray')#, vmax=y_pred_mat.max())
axes[1].set_title("Design matrix")
utils.add_cbar(fig, axes[1])

axes[2].imshow(adj, cmap='gray')#, vmax=y_pred_mat.max())
axes[2].set_title("Effective delays $x=\mathbf{1}$ (if bundle)")
utils.add_cbar(fig, axes[2])

nx.draw(toy_graph, ax=axes[3], with_labels=True)

### Backward pass: Jointly optimizing effective delays and alphas

In [ ]:
max_path_depth = 4
multi_hops_design = regmod.get_path_matrices(adj, max_path_depth)

In [ ]:
x_ground = utils.remove_diagonal_entries(adj).flatten()

a = 0
# Computes the alpha vector as alpha_n = a^n
alpha = solver.torch.tensor([a**(i+1) for i in range(max_path_depth)])

design_model = solver.combine_paths_matrices_torch(solver.torch.tensor(multi_hops_design), alpha=alpha)
y_pred = solver.forward(design_model, solver.torch.tensor(x_ground))

In [ ]:
from copy import deepcopy
np.random.seed(99)
y_ground = solver.torch.tensor(deepcopy(y_pred))
x_init = solver.torch.tensor(np.random.rand(len(x_ground))).requires_grad_(True)
# alphas_init = solver.torch.tensor(np.random.rand(len(alpha))).requires_grad_(True)

In [ ]:
importlib.reload(solver)
x = deepcopy(x_init)
x_opt, loss = solver.naive_gradient_descent(x, y_ground, 0, solver.torch.tensor(multi_hops_design),
                                            n_iter=1000, verbose=False, early_stop=1e-10)

In [ ]:
x_ground_mat = utils.add_diagonal_entries(x_ground.reshape(adj.shape[0], adj.shape[1]-1))
x_pred_mat = utils.add_diagonal_entries(x_opt.reshape(adj.shape[0], adj.shape[1]-1))

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
fig.suptitle(rf"Path design matrix, $\alpha=$"+" ".join([f"{a:1.4f}" for a in list(alpha)]))
axes[0].imshow(x_ground_mat, cmap='gray')
axes[0].set_title("Effective delays $x=\mathbf{1}$ (if bundle)")
utils.add_cbar(fig, axes[0])
utils.annotate_heatmap(fig, axes[0], x_ground_mat, adapt_color=0.6)

axes[1].imshow(x_pred_mat, cmap='gray')#, vmax=y_pred_mat.max())
axes[1].set_title(f"Estimated Effective delays w/ loss={np.round(loss,4)}")
utils.add_cbar(fig, axes[1])
# NOTE: we need to rechek this, it seems that the colors are flipped? (transposed?)
utils.annotate_heatmap(fig, axes[1], x_pred_mat.T, adapt_color=0.6)

### Backward: Pseudo-inverse solution

In [ ]:
max_path_depth = 4
multi_hops_design = regmod.get_path_matrices(adj, max_path_depth)

In [ ]:
x_ground = utils.remove_diagonal_entries(adj).flatten()

a = 0
# Computes the alpha vector as alpha_n = a^n
alpha = solver.torch.tensor([a**(i+1) for i in range(max_path_depth)])

design_model = solver.combine_paths_matrices_torch(solver.torch.tensor(multi_hops_design), alpha=alpha)
design_model = solver.torch.tensor(design_shortest)
y_pred = solver.forward(design_model, solver.torch.tensor(x_ground)).numpy()

In [ ]:
x_opt = solver.pseudo_inverse(y_pred, a_design=design_model.numpy())
y_est = design_model.numpy() @ x_opt
loss = np.linalg.norm(y_est - y_pred)

In [ ]:
x_ground_mat = utils.add_diagonal_entries(x_ground.reshape(adj.shape[0], adj.shape[1]-1))
x_pred_mat = utils.add_diagonal_entries(x_opt.reshape(adj.shape[0], adj.shape[1]-1))

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
fig.suptitle(rf"Path design matrix, $\alpha=$"+" ".join([f"{a:1.4f}" for a in list(alpha)]))
axes[0].imshow(x_ground_mat, cmap='gray')
axes[0].set_title("Effective delays $x=\mathbf{1}$ (if bundle)")
utils.add_cbar(fig, axes[0])
utils.annotate_heatmap(fig, axes[0], x_ground_mat, adapt_color=0.6)

axes[1].imshow(x_pred_mat, cmap='gray')#, vmax=y_pred_mat.max())
axes[1].set_title(f"Estimated Effective delays w/ loss={np.round(loss,4)}")
utils.add_cbar(fig, axes[1])
utils.annotate_heatmap(fig, axes[1], np.round(x_pred_mat,4), adapt_color=0.6)

## F-tract Example

Contents:
- Sanity check 
    - generating measured delay by considering only 1s delay for all bundles
    - verifying that the solver regresses back the only 1 delays

- Regressing conductance delays

In [ ]:
import importlib
importlib.reload(solver)

In [ ]:
path_to_data = "../resources"

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

filename = "adjacency_atlas.pkl"

adj = utils.load(op.join(path_to_data, filename))
# TODO + NOTE: temporary truncation to remove (by michael)
adj = adj[:84, :84] 
adj -= np.diag(np.diag(adj))

axes[0].imshow(adj, cmap='gray')
axes[0].set_title('Structural connectivity')
toy_graph = nx.Graph(adj)
nx.draw(toy_graph, ax=axes[1], with_labels=True)

In [ ]:
importlib.reload(regmod)
max_path_depth = 3

#multi_hops_design = regmod.get_path_matrices(adj, max_path_depth)

#a = 0
# Computes the alpha vector as alpha_n = a^n
#alpha = [a**(i+1) for i in range(max_path_depth)]
#alpha = a

#design_shortest = regmod.combine_paths_matrices(multi_hops_design, alpha=alpha)
design_shortest = regmod.build_design_shortest(adj, n_subopt=1, alpha=0.5)

y_pred_mat = regmod.predict_conduction_delays(design_shortest, adj, invert_weights=False)

fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(20, 5))
fig.suptitle(rf"Path design matrix, $\alpha=$"+" ".join([f"{a:1.4f}" for a in list(alpha)]))
axes[0].imshow(y_pred_mat, cmap='hot', interpolation=None)
axes[0].set_title("Conductance delays $\hat{y}$")
utils.add_cbar(fig, axes[0])
#utils.annotate_heatmap(fig, axes[0], y_pred_mat, adapt_color=0.6)

axes[1].imshow(design_shortest, cmap='gray')#, vmax=y_pred_mat.max())
axes[1].set_title("Design matrix")
utils.add_cbar(fig, axes[1])

axes[2].imshow(adj, cmap='gray')#, vmax=y_pred_mat.max())
axes[2].set_title("Effective delays $x=\mathbf{1}$ (if bundle)")
utils.add_cbar(fig, axes[2])

nx.draw(toy_graph, ax=axes[3], with_labels=True)

### Sanity check

In [ ]:
n_reduced = -1
reduced_adj = adj[:n_reduced][:, :n_reduced]

x_ground = utils.remove_diagonal_entries(reduced_adj).flatten()

a = 0.5
# Computes the alpha vector as alpha_n = a^n
alpha = solver.torch.tensor([a**(i+1) for i in range(max_path_depth)])

#design_model = solver.combine_paths_matrices_torch(solver.torch.tensor(multi_hops_design), alpha=alpha)

design_shortest = regmod.build_design_shortest(reduced_adj, n_subopt=1, alpha=a)
design_model = solver.torch.tensor(design_shortest)
x_ground = utils.remove_diagonal_entries(reduced_adj).flatten()
y_pred = solver.forward(design_model, solver.torch.tensor(x_ground)).numpy()

In [ ]:
x_opt = solver.pseudo_inverse(y_pred, a_design=design_model.numpy(), rcond=1e-10)
y_est = design_model.numpy() @ x_opt
loss = np.linalg.norm(y_est - y_pred)

In [ ]:
x_ground_mat = utils.add_diagonal_entries(x_ground.reshape(reduced_adj.shape[0], reduced_adj.shape[1]-1))
x_pred_mat = utils.add_diagonal_entries(x_opt.reshape(reduced_adj.shape[0], reduced_adj.shape[1]-1))

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
fig.suptitle(rf"Path design matrix, $\alpha=$"+" ".join([f"{a:1.4f}" for a in list(alpha)]))
axes[0].imshow(x_ground_mat, cmap='gray')
axes[0].set_title("Effective delays $x=\mathbf{1}$ (if bundle)")
utils.add_cbar(fig, axes[0])
# utils.annotate_heatmap(fig, axes[0], x_ground_mat, adapt_color=0.6)

axes[1].imshow(x_pred_mat, cmap='gray')#, vmax=y_pred_mat.max())
axes[1].set_title(f"Estimated Effective delays w/ loss={np.round(loss,4)}")
utils.add_cbar(fig, axes[1])
# utils.annotate_heatmap(fig, axes[1], np.round(x_pred_mat,4), adapt_color=0.6)

In [ ]:
fig, ax = plt.subplots()
#ax.imshow(np.abs(x_pred_mat - x_ground_mat) > 0.1)
ax.imshow(x_pred_mat - x_ground_mat, vmin=-1e-5, vmax=1e-5)
ax.set_title('Difference (>10%) map between predicted and ground truth matrices')

### From conductance delay

In [ ]:
n_reduced = 20

y_ground_mat = utils.load("../resources/conductance-delay_Lausanne2008_33.pkl")
y_ground_mat = np.nan_to_num(y_ground_mat['median'])[:n_reduced][:, :n_reduced]

y_ground = utils.remove_diagonal_entries(y_ground_mat).flatten()

In [ ]:
adj_reduced = adj[:n_reduced][:, :n_reduced]

a = 0
# Computes the alpha vector as alpha_n = a^n
#alpha = solver.torch.tensor([a**(i+1) for i in range(max_path_depth)])

#design_model = solver.combine_paths_matrices_torch(solver.torch.tensor(multi_hops_design), alpha=alpha).numpy()
design_shortest = regmod.build_design_shortest(adj_reduced, n_subopt=0, alpha=a)
design_model = solver.torch.tensor(design_shortest)

In [ ]:
x_opt = solver.pseudo_inverse(y_ground, a_design=design_model, rcond=1e-10)
y_est = design_model @ x_opt
loss = np.linalg.norm(y_est - y_ground)

In [ ]:
x_pred_mat = utils.add_diagonal_entries(x_opt.reshape(adj_reduced.shape[0], adj_reduced.shape[1]-1))

fig, axes = plt.subplots(ncols=3, figsize=(15, 5))
fig.suptitle(rf"Path design matrix, $\alpha=$"+" ".join([f"{a:1.4f}" for a in list(alpha)]))

axes[0].imshow(y_ground_mat, cmap='gray')
axes[0].set_title(f"Conduction delays $y$")
utils.add_cbar(fig, axes[0])
# utils.annotate_heatmap(fig, axes, np.round(x_pred_mat,4), adapt_color=0.6)

axes[1].imshow(x_pred_mat, cmap='gray')
axes[1].set_title(f"Estimated Conduction delays $\hat y$ w/ loss={np.round(loss,4)}")
utils.add_cbar(fig, axes[1])

axes[2].imshow(x_pred_mat, cmap='gray', vmin=0)
axes[2].set_title(f"Clipped $\hat y$")
utils.add_cbar(fig, axes[2])

In [ ]:
diff = y_ground_mat - np.clip(x_pred_mat, a_min=0, a_max=None)
maxval = np.abs(diff).max()
plt.imshow(diff, vmin=-maxval, vmax=maxval, cmap="coolwarm")
plt.colorbar()

# Visualize the influence of $\alpha$

In [ ]:
n_reduced = 50

y_ground_mat = utils.load("../resources/conductance-delay_Lausanne2008_33.pkl")
y_ground_mat = np.nan_to_num(y_ground_mat['median'])[:n_reduced][:, :n_reduced]

y_ground = utils.remove_diagonal_entries(y_ground_mat).flatten()

adj_reduced = adj[:n_reduced][:, :n_reduced]

n_alphas=10
alphas = np.linspace(0, 2, n_alphas)

losses = np.zeros_like(alphas)
all_sol = np.zeros((len(alphas), adj_reduced.shape[0], adj_reduced.shape[1]))

for a_i, a in enumerate(alphas):
    design_shortest = solver.torch.tensor(regmod.build_design_shortest(adj_reduced, n_subopt=1, alpha=a))

    x_opt = solver.pseudo_inverse(y_ground, a_design=design_shortest, rcond=1e-10)
    all_sol[a_i] = utils.add_diagonal_entries(x_opt.reshape(adj_reduced.shape[0], adj_reduced.shape[1]-1))

    y_est = design_shortest @ x_opt
    losses[a_i] = np.linalg.norm(y_est - y_ground)


In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(18, 5))

axes[0].plot(alphas, losses)
axes[0].set_title(r"Loss as a function of $\alpha$")

axes[1].imshow(y_ground_mat, cmap='gray')
axes[1].set_title(r"Original conductance delays $y$")
utils.add_cbar(fig, axes[1])

axes[2].imshow(all_sol[np.argmin(losses)], cmap='gray')
axes[2].set_title(rf"Clipped effective delays $x^*$ for $\alpha={alphas[np.argmin(losses)]}$")
utils.add_cbar(fig, axes[2])